# Parser Gold Mapping

In [16]:
import json
import numpy as np
import conll16st.aligner as aligner
import conll16st.partial_scorer as ps
import read_write_files as rw

In [143]:
def read_json(file_path):
    return [json.loads(x) for x in open(file_path)]

In [144]:
def save_json(alignments,file_path):
    with open(file_path, 'w') as f:
        for alignment in alignments:
            f.write(json.dumps(alignment)+"\n")

In [130]:
gold_path = "data/gold_standard/blind/gold.json"
gold_list = rw.read_json(gold_path)

In [131]:
predicted_lists = []

In [132]:
parse_path = "data/submissions/randomized/oslopots.json"
predicted_list = rw.read_json(parse_path)
predicted_lists += [("oslopots",predicted_list)]

In [133]:
parse_path = "data/submissions/randomized/nguyenlab.json"
predicted_list = rw.read_json(parse_path)
predicted_lists += [("nguyenlab",predicted_list)]

In [134]:
def change_sets_to_list(alignments):
    for gold,pred in alignments:
        if gold != None:
            gold["Arg1"]["TokenIndexSet"] = list(gold["Arg1"]["TokenIndexSet"])
            gold["Arg2"]["TokenIndexSet"] = list(gold["Arg2"]["TokenIndexSet"])
        if pred != None:
            pred["Arg1"]["TokenIndexSet"] = list(pred["Arg1"]["TokenIndexSet"])
            pred["Arg2"]["TokenIndexSet"] = list(pred["Arg2"]["TokenIndexSet"])
        
    return alignments

In [147]:
def align_parsers_to_gold(gold_rel,parsers):
    total_alignment = {gold["ID"]:{"gold":gold,"parsers":[]} for gold in gold_rel}
    parsers_not_mappable = []
    
    parser_names = [name for name,parser in parsers]
    
    for name,parser_relations in parsers:
        arg1_alignment, arg2_alignment, relation_alignment = aligner.align_relations(
            gold_rel, 
            parser_relations, 
            0.7)
         
        relation_alignment = change_sets_to_list(relation_alignment)
        for gold_align,pred_align in relation_alignment:
            if gold_align == None:
                parsers_not_mappable += [pred_align]
            else:
                total_alignment[gold_align["ID"]]["parsers"] += [pred_align]
                total_alignment[gold_align["ID"]]["parser_names"] = parser_names
     
    return total_alignment,parsers_not_mappable
        

In [148]:
total_alignment,not_mappables = align_parsers_to_gold(
    gold_rel=gold_list,
    parsers=predicted_lists)

In [149]:
save_json(total_alignment.values(),"data/project_files/total_alignment.json")
save_json(not_mappables,"data/project_files/not_mappable.json")